In [82]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
from timeit import default_timer as timer
import datetime
# from utils import summary_stats, stats_with_description, safe_last, match_gvkey, merge_df, find_consec_year, safe_div
# import requests
# from bs4 import BeautifulSoup
# from soup2dict import convert
# from urllib.parse import urljoin, urlparse
# import matplotlib.pyplot as plt
# import seaborn as sns
# import matplotlib.dates as mdates
# import matplotlib.ticker as mtick
# import time
# import zipfile
# import pyreadstat
# import itertools
from thefuzz import fuzz
from thefuzz import process
import re
import joblib
import zipfile


# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# set folders
CHECKPOINT_FOLDER = '.\\Checkpoints'
CODING_TABLES_FOLDER = '.\\Coding_tables'
STATS_FOLDER = '.\\Stats'


if not os.path.exists(CHECKPOINT_FOLDER):
    os.makedirs(CHECKPOINT_FOLDER)
if not os.path.exists(STATS_FOLDER):
    os.makedirs(STATS_FOLDER)

In [4]:
df_fundam_final = pd.read_pickle(os.path.join(CHECKPOINT_FOLDER, 'df_fundam_final.pkl'))

## Read file with CDS intensity
Downloaded from  # todo

In [105]:
df_CDS_intensity=pd.read_csv(os.path.join(CODING_TABLES_FOLDER, 'hazard_05to18.csv'))
df_CDS_intensity['Date']= pd.to_datetime(df_CDS_intensity['Date'])
df_CDS_intensity

,index,Date,Ticker,DocClause,Recovery,CompositeLevelRecovery,Risk Intensity 6m,Risk Intensity 1y,Risk Intensity 2y,Risk Intensity 3y,Risk Intensity 4y,Risk Intensity 5y,Risk Intensity 7y,Risk Intensity 10y,Risk Intensity 15y,Risk Intensity 20y,Risk Intensity 30y
0,0,2018-10-31,AES,XR14,0.40000,CcyGrp,0.001716,0.003462,0.008852,0.019450,0.033265,0.050764,0.033289,0.028931,0.044469,0.031585,0.015457
1,1,2018-10-31,AES,XR14,0.20000,CcyGrp,0.001287,0.002571,0.006599,0.014421,0.024505,0.037097,0.024731,0.021693,0.032368,0.024032,0.013311
2,2,2018-10-31,AMD,XR14,0.39714,CcyGrp,0.003167,0.003902,0.010246,0.024054,0.040481,0.066788,0.067063,0.050646,0.046717,0.047090,0.036739
3,3,2018-10-31,BBY,XR14,0.40000,CcyGrp,0.001316,0.001719,0.004699,0.012619,0.024388,0.036903,0.047071,0.036017,0.028172,0.029957,0.025853
4,4,2018-10-31,BBY,XR14,0.26667,CcyGrp,0.001391,0.001858,0.005113,0.013692,0.025209,0.037981,0.048760,0.037740,0.030104,0.032260,0.028336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102892,102892,2005-03-01,SVU,XR14,0.39722,CcyGrp,0.000863,0.000796,0.003157,0.005900,NaN,0.009078,0.010357,0.015101,0.009755,0.016322,0.007404
102893,102893,2005-03-01,T,XR14,0.39444,CcyGrp,0.001717,0.004212,0.009209,0.013362,NaN,0.029835,0.033091,0.032609,0.028536,0.023898,0.029379
102894,102894,2005-03-01,THC,XR14,0.40000,CcyGrp,0.018059,0.028004,0.046942,0.063762,NaN,0.076099,0.066806,0.077485,0.055536,0.068227,0.042305
102895,102895,2005-03-01,UNH,XR14,0.39808,Entity Tier,0.000648,0.000916,0.003382,0.003145,NaN,0.005325,0.008152,0.010025,0.004811,0.015523,0.007320


In [39]:
# check available tickers
ticker_fundam=df_fundam_final['tic'].unique()
ticker_CDS=df_CDS_intensity['Ticker'].unique()
print(f'- Total ticker in "df_fundam_final": {len(ticker_fundam)}')
print(f'- Total ticker in "df_CDS_intensity": {len(ticker_CDS)}')
match_ticker=set(ticker_fundam) & set(ticker_CDS)
print(f'- Common ticker: {len(match_ticker)}')

- Total ticker in "df_fundam_final": 23425
- Total ticker in "df_CDS_intensity": 26
- Common ticker: 24


In [106]:
df_CDS_intensity[(df_CDS_intensity['Ticker']=='BBY') & (df_CDS_intensity['Date']=='2005-01-02')]

,index,Date,Ticker,DocClause,Recovery,CompositeLevelRecovery,Risk Intensity 6m,Risk Intensity 1y,Risk Intensity 2y,Risk Intensity 3y,Risk Intensity 4y,Risk Intensity 5y,Risk Intensity 7y,Risk Intensity 10y,Risk Intensity 15y,Risk Intensity 20y,Risk Intensity 30y
102413,102413,2005-01-02,BBY,XR14,0.40,CcyGrp,0.00095,0.002495,0.004159,0.005984,NaN,0.009779,0.011520,0.014624,0.007536,0.019851,NaN
102414,102414,2005-01-02,BBY,XR14,0.25,CcyGrp,NaN,0.002119,0.005812,0.006347,NaN,0.013639,0.013464,0.015926,0.011807,-0.002914,NaN


In [32]:
df_CDS_intensity.groupby(['Date', 'Ticker'], as_index=False).size().query('size > 1')

,Date,Ticker,size
2,2005-01-02,BBY,2
10,2005-01-02,JPM,2
22,2005-01-03,BBY,2
30,2005-01-03,JPM,2
39,2005-01-04,AES,2
...,...,...,...
90546,2018-12-09,JPM,2
90560,2018-12-10,AES,2
90562,2018-12-10,BBY,2
90569,2018-12-10,INTC,2


## Read file with CDS spread
Downloaded from Markit

Data dictionary: https://datacenter.safefrankfurt.de/datacenter/documents/CDS%20Data%20Dictionary%20and%20Automating%20Report%20Downloads.pdf

In [103]:
ZIP_FILE=r'C:\Users\Alessandro Bitetto\Desktop\MARKIT data\CDSCOMP.zip'
DOCCLAUSE='XR14'
CURRENCY='USD'
TIER='SNRFOR'

zf = zipfile.ZipFile(ZIP_FILE)
zip_file_content=zf.namelist()
zip_file_content=[x for x in zip_file_content if '.csv' in x]
df_CDS_spread=pd.DataFrame()
start = timer()
check_point_list=[]
for i, file in enumerate(zip_file_content):
    
    print('Extracting ' + str(i + 1) + ' / ' + str(len(zip_file_content)), end = '\r')
    
    df_t=pd.read_csv(zf.open(file), sep=',', low_memory=False, skiprows=[0,1])
    df_t=df_t[df_t['DocClause'] == DOCCLAUSE]
    df_t=df_t[df_t['Ccy'] == CURRENCY]
    df_t=df_t[df_t['Tier'] == TIER]
    df_CDS_spread=pd.concat([df_CDS_spread, df_t])
    
    # save checkpoint
    if i % 300 == 0 or i == (len(zip_file_content) - 1):
        chk_path=os.path.join(CHECKPOINT_FOLDER, 'df_CDS_spread_temp'+str(i)+'.pkl')
        check_point_list.append(chk_path)
        df_CDS_spread.to_pickle(chk_path, protocol=-1)
        df_CDS_spread=pd.DataFrame()

# recover full dataset
df_CDS_spread=pd.DataFrame()
for chk_path in check_point_list:
    df_CDS_spread=pd.concat([df_CDS_spread, pd.read_pickle(chk_path)])
check_rows=df_CDS_spread.Date.nunique()
if check_rows != len(zip_file_content):
    print(f'- Warning: expected read file number {len(zip_file_content)} mismatch: {check_rows} found')
for chk_path in check_point_list:
    os.remove(chk_path)
        
df_CDS_spread.to_pickle(os.path.join(CHECKPOINT_FOLDER, 'df_CDS_spread.pkl'), protocol=-1)
print('\n- Pickle saved to', os.path.join(CHECKPOINT_FOLDER, 'df_CDS_spread.pkl'))

print('\nTotal elapsed time:', str(datetime.timedelta(seconds=round(timer()-start))))

Extracting 4652 / 4652
- Pickle saved to .\Checkpoints\df_CDS_spread.pkl

Total elapsed time: 0:11:58


In [108]:
df_CDS_spread.shape

(2479155, 50)

In [ ]:
df_CDS_spread=pd.read_pickle(os.path.join(CHECKPOINT_FOLDER, 'df_CDS_spread.pkl'))

In [ ]:
summary_stats

In [96]:
# check available tickers
ticker_fundam=df_fundam_final['tic'].unique()
ticker_CDS=df_CDS_spread['Ticker'].unique()
print(f'- Total ticker in "df_fundam_final": {len(ticker_fundam)}')
print(f'- Total ticker in "df_CDS_intensity": {len(ticker_CDS)}')
match_ticker=set(ticker_fundam) & set(ticker_CDS)
print(f'- Common ticker: {len(match_ticker)}')

- Total ticker in "df_fundam_final": 23425
- Total ticker in "df_CDS_intensity": 854
- Common ticker: 342


In [ ]:
df_CDS_spread.groupby(['Ticker']).agg(Tot_name=('ShortName', lambda x: len(np.unique(x)))).query('Tot_name > 1')

In [ ]:
df_CDS_spread.groupby(['ShortName']).agg(Tot_ticker=('Ticker', lambda x: len(np.unique(x)))).query('Tot_ticker > 1')

In [ ]:
# try to match by company name

start = timer()
choices=df_fundam_final['conm'].unique().tolist()
CDS_name=df_CDS_spread[['Ticker', 'ShortName']].drop_duplicates()
CDS_name[['conm', 'accuracy']]=CDS_name['ShortName'].apply(lambda x: pd.Series(process.extractOne(x, choices)))

CDS_name.to_pickle(os.path.join(CHECKPOINT_FOLDER, 'CDS_name.pkl'), protocol=-1)
print('\n- Pickle saved to', os.path.join(CHECKPOINT_FOLDER, 'df_CDS_spread.pkl'))
print('\nTotal elapsed time:', str(datetime.timedelta(seconds=round(timer()-start))))

In [ ]:
CDS_name=pd.read_pickle(os.path.join(CHECKPOINT_FOLDER, 'CDS_name.pkl'))